# Docling PDF Reader — Document-native RAG

In [ ]:
from rich.pretty import pprint
import warnings
import os
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings(
    action="ignore", category=FutureWarning, module="easyocr"
)
warnings.filterwarnings(
    action="ignore", category=UserWarning, module="pydantic"
)

## Basic usage

To leverage Docling's rich native format, we configure `export_type` to `"json"`.

In [ ]:
from llama_index.readers.docling import DoclingPDFReader

reader = DoclingPDFReader(export_type="json")
docs = reader.load_data(
    file_path="https://arxiv.org/pdf/2408.09869",  # PDF local path or URL (or iterable thereof)
)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In order to appropriately parse / chunk the Docling native format of the above-generated documents, we use a `DoclingNodeParser`:

In [ ]:
from llama_index.node_parser.docling import DoclingNodeParser

node_parser = DoclingNodeParser()
nodes = node_parser.get_nodes_from_documents(documents=docs)

Notice how nodes contain not only the textual information, but also metadata like page, bounding box etc:

In [ ]:
pprint(nodes[5], max_length=5, max_string=200, max_depth=2)

TextNode(
│   id_='ea8d6ee2-9443-4787-82a1-b025d7c9c70c',
│   embedding=None,
│   metadata={
│   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   'path': '#/main-text/21',
│   │   'heading': '2 Getting Started',
│   │   'page': 2,
│   │   'bbox': [...]
│   },
│   excluded_embed_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   excluded_llm_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...),
│   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(...),
│   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(...)
│   },
│   text='To use Docling, you can simply install the docling package from PyPI. Documentation and examples are available in our GitHub repository at github.com/DS4SD/docling. All required model assets 1 are dow'+121,
│   mimetype='text/plain',
│   start_char_idx=8531,
│   end_char_idx=8852,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

We now define the embed model:

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

We set up the vector store:

In [ ]:
from tempfile import TemporaryDirectory
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri=os.environ.get(
        "MILVUS_URI",
        default=f"{(tmp_dir := TemporaryDirectory()).name}/milvus_demo.db",
    ),
    collection_name="docling_collection",
    dim=len(embed_model.get_text_embedding("hi")),
    overwrite=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


We create the `VectorStoreIndex`, triggering the encoding of the chunks and the ingestion into the vector store:

In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes=nodes,
    embed_model=embed_model,
    storage_context=storage_context,
    show_progress=True,
)

Generating embeddings:   0%|          | 0/83 [00:00<?, ?it/s]

Last but not least, we set up the LLM:

In [ ]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

HF_TOKEN = os.environ.get("HF_TOKEN")

llm = HuggingFaceInferenceAPI(
    token=HF_TOKEN,
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
)

With all pieces in place, we are now ready to ask questions on our document content.

As shown below, besides the response itself, we are also getting document-native grounding, incl. page number and bounding box information:

In [ ]:
query_engine = index.as_query_engine(llm=llm)
query_res = query_engine.query("Which are the main AI models in Docling?")
pprint(query_res, max_length=5, max_string=170, max_depth=5)

Response(
│   response=' The main AI models in Docling are a layout analysis model, which is an accurate object-detector for page elements, and TableFormer, a state-of-the-art table structure re'+16,
│   source_nodes=[
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='e52dd847-6f12-40ec-83d1-33ca62778b78',
│   │   │   │   embedding=None,
│   │   │   │   metadata={
│   │   │   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   │   │   'path': '#/main-text/36',
│   │   │   │   │   'heading': '3.2 AI models',
│   │   │   │   │   'page': 3,
│   │   │   │   │   'bbox': [...]
│   │   │   │   },
│   │   │   │   excluded_embed_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   excluded_llm_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   relationships={
│   │   │   │   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...),
│   │   │   │   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(...),
│   │   │   │   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(...)
│   │   │   │   },
│   │   │   │   text='As part of Docling, we initially release two highly capable AI models to the open-source community, which have been developed and published recently by our team. The firs'+438,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=14945,
│   │   │   │   end_char_idx=15553,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.8484227061271667
│   │   ),
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='4a40ac2d-e39e-422a-a5c3-97aa04ab8aae',
│   │   │   │   embedding=None,
│   │   │   │   metadata={
│   │   │   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   │   │   'path': '#/main-text/10',
│   │   │   │   │   'heading': '1 Introduction',
│   │   │   │   │   'page': 1,
│   │   │   │   │   'bbox': [...]
│   │   │   │   },
│   │   │   │   excluded_embed_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   excluded_llm_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   │   │   │   relationships={
│   │   │   │   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...),
│   │   │   │   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(...),
│   │   │   │   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(...)
│   │   │   │   },
│   │   │   │   text='With Docling , we open-source a very capable and efficient document conversion tool which builds on the powerful, specialized AI models and datasets for layout analysis a'+318,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=5261,
│   │   │   │   end_char_idx=5749,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.7966917753219604
│   │   )
│   ],
│   metadata={
│   │   'e52dd847-6f12-40ec-83d1-33ca62778b78': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'path': '#/main-text/36',
│   │   │   'heading': '3.2 AI models',
│   │   │   'page': 3,
│   │   │   'bbox': [107.25257873535156, 330.1919250488281, 504.05230712890625, 406.1560974121094]
│   │   },
│   │   '4a40ac2d-e39e-422a-a5c3-97aa04ab8aae': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'path': '#/main-text/10',
│   │   │   'heading': '1 Introduction',
│   │   │   'page': 1,
│   │   │   'bbox': [107.16082763671875, 83.33924865722656, 504.09375, 136.9646759033203]
│   │   }
│   }
)